# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-pahg203m
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-pahg203m
  Created wheel for Theano: filename=Theano-1.0.4+23.g630974a7b-cp37-none-any.whl size=2667498 sha256=ee005558c3d76cecf511fa622175942a5265d032a60db253451a221a2d7b7891
  Stored in directory: /tmp/pip-ephem-wheel-cache-l242gg26/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+23.g630974a7b
    Uninstalling Theano-1.0.4+23.g630974a7b:
      Successfully uninstalled Theano-1.0.4+23.g630974a7b
Requirement already up-to-date: tensorflow in /opt/conda/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: keras in /opt/conda/lib/python3.7/site-packages (2.3.1)
Requirement already up-to-date: pip in /opt/conda/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: pandas in /opt/conda/lib/python3.7/site-packages (

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 240us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 197us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 3s 339us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 3s 354us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 3s 348us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 3s 378us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 3s 385us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 3s 370us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 3s 375us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 3s 381us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 78/100
8000/8000 [==============================] - 3s 389us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 79/100
8000/8000 [==============================] - 1s 162us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 1s 142us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 1s 139us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 1s 163us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 2s 202us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 2s 214us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 2s 257us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 1

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [12]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 4s 459us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 3s 394us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 3s 395us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 3s 398us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 3s 397us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 3s 405us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 3s 404us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 3s 408us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 3s 423us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 3s 371us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 79/100
8000/8000 [==============================] - 3s 340us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 3s 338us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 3s 314us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 3s 334us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 2s 288us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 2s 253us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 2s 282us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 2s 243us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 87/100
8000/8000 [==============================] - 2

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [15]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 162us/step - loss: 0.6136 - accuracy: 0.7945
Epoch 2/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.5335 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 230us/step - loss: 0.5118 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.5069 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 176us/step - loss: 0.5061 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 2s 240us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 2s 188us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 166us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 1s 148us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 79/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 1s 183us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 1s 181us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 1s 177us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 1s 157us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 1s 161us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 2s 251us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 87/100
8000/8000 [==============================] - 2

In [16]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [19]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()


# Tuning the ANN

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [21]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 2s 236us/step - loss: 0.4886 - accuracy: 0.7969
Epoch 2/50
7200/7200 [==============================] - 2s 228us/step - loss: 0.4273 - accuracy: 0.8057
Epoch 3/50
7200/7200 [==============================] - 2s 268us/step - loss: 0.4200 - accuracy: 0.8285
Epoch 4/50
7200/7200 [==============================] - 1s 190us/step - loss: 0.4213 - accuracy: 0.8281
Epoch 5/50
7200/7200 [==============================] - 2s 216us/step - loss: 0.4183 - accuracy: 0.8292
Epoch 6/50
7200/7200 [==============================] - 1s 187us/step - loss: 0.4205 - accuracy: 0.8296
Epoch 7/50
7200/7200 [==============================] - 1s 178us/step - loss: 0.4191 - accuracy: 0.8286
Epoch 8/50
7200/7200 [==============================] - 2s 223us/step - loss: 0.4179 - accuracy: 0.8288
Epoch 9/50
7200/7200 [==============================] - 1s 177us/step - loss: 0.4162 - accuracy: 0.8318
Epoch 10/50
7200/7200 [==============================] - 1s 197u

7200/7200 [==============================] - 1s 191us/step - loss: 0.4135 - accuracy: 0.8286
Epoch 30/50
7200/7200 [==============================] - 1s 186us/step - loss: 0.4117 - accuracy: 0.8294
Epoch 31/50
7200/7200 [==============================] - 1s 189us/step - loss: 0.4106 - accuracy: 0.8326
Epoch 32/50
7200/7200 [==============================] - 1s 180us/step - loss: 0.4118 - accuracy: 0.8356
Epoch 33/50
7200/7200 [==============================] - 1s 188us/step - loss: 0.4105 - accuracy: 0.8321
Epoch 34/50
7200/7200 [==============================] - 1s 186us/step - loss: 0.4119 - accuracy: 0.8310
Epoch 35/50
7200/7200 [==============================] - 1s 183us/step - loss: 0.4099 - accuracy: 0.8354
Epoch 36/50
7200/7200 [==============================] - 1s 184us/step - loss: 0.4100 - accuracy: 0.8311
Epoch 37/50
7200/7200 [==============================] - 1s 190us/step - loss: 0.4105 - accuracy: 0.8332
Epoch 38/50
7200/7200 [==============================] - 1s 190us/s

KeyboardInterrupt: 

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)